# **Chapter 2: Next-Generation Sequencing**
</br>

**Performing basic sequence analysis**

**Goal:** working with *FASTA* files and do some manipulation, such as reverse complementing or transcription. 
</br>
We will use the **human Lactase (LCT) gene** as an example.

firstly, we need to import *Entrez, SeqIO* from **BioPython** package.
</br>
`Entrez` : for search and fetch data from NCBI databases
</br>
`SeqIO` : Sequence input/output
</br>
**Note:** Do not forget to put in the correct email address.

In [ ]:
# pip install biopython # on google colab
conda install biopython # on galaxy server

In [1]:
from Bio import Entrez, SeqIO, SeqRecord
Entrez.email = 'your@email.here'

**how we can search and fetch data from NCBI databases?**

**First Step:** search on the nucleotide database
</br>
to do that, we should use: `Entrez.esearch()` function
</br>
`Entrez.esearch()` arguments: 
1.   db: database name
2.   term: ncbi search query (for more detail, see: https://www.ncbi.nlm.nih.gov/nuccore/advanced)
3. rettype: Retrieval type


In [ ]:
hdl = Entrez.efetch(db='nucleotide', id=['NM_002299'], rettype='gb') #Lactase gene
gb_rec = SeqIO.read(hdl, 'gb')

**Next Step:** read the result that is returned
</br>
to do that, we should use: `SeqIO.read()` function
</br>
outputs of `Entrez.read()` function:

In [78]:
print(gb_rec)
# only sequence
print(gb_rec.seq)

ID: NM_002299.4
Name: NM_002299.4
Description: NM_002299.4 Homo sapiens lactase (LCT), mRNA
Number of features: 0
Seq('AACAGTTCCTAGAAAATGGAGCTGTCTTGGCATGTAGTCTTTATTGCCCTGCTA...GTC')
AACAGTTCCTAGAAAATGGAGCTGTCTTGGCATGTAGTCTTTATTGCCCTGCTAAGTTTTTCATGCTGGGGGTCAGACTGGGAGTCTGATAGAAATTTCATTTCCACCGCTGGTCCTCTAACCAATGACTTGCTGCACAACCTGAGTGGTCTCCTGGGAGACCAGAGTTCTAACTTTGTAGCAGGGGACAAAGACATGTATGTTTGTCACCAGCCACTGCCCACTTTCCTGCCAGAATACTTCAGCAGTCTCCATGCCAGTCAGATCACCCATTATAAGGTATTTCTGTCATGGGCACAGCTCCTCCCAGCAGGAAGCACCCAGAATCCAGACGAGAAAACAGTGCAGTGCTACCGGCGACTCCTCAAGGCCCTCAAGACTGCACGGCTTCAGCCCATGGTCATCCTGCACCACCAGACCCTCCCTGCCAGCACCCTCCGGAGAACCGAAGCCTTTGCTGACCTCTTCGCCGACTATGCCACATTCGCCTTCCACTCCTTCGGGGACCTAGTTGGGATCTGGTTCACCTTCAGTGACTTGGAGGAAGTGATCAAGGAGCTTCCCCACCAGGAATCAAGAGCGTCACAACTCCAGACCCTCAGTGATGCCCACAGAAAAGCCTATGAGATTTACCACGAAAGCTATGCTTTTCAGGGCGGAAAACTCTCTGTTGTCCTGCGAGCTGAAGATATCCCGGAGCTCCTGCTAGAACCACCCATATCTGCGCTTGCCCAGGACACGGTCGATTTCCTCTCTCTTGATTTGTCTTATGAATGCCAAAATGAGGCAAGTCTGCGGCAGAAGCTGAGTAAATTGCA

In [79]:
for feature in gb_rec.features:    
    if feature.type == 'CDS':
        location = feature.location  # Note translation existing 
cds = SeqRecord.SeqRecord(gb_rec.seq[location.start:location. end], 'NM_002299', description='LCT CDS only')

CDS: coding sequence

let's start by **saving it to a FASTA file on our local disk**:

In [71]:
from Bio import SeqIO
w_hdl = open('example.fasta', 'w')
SeqIO.write([cds], w_hdl, 'fasta')
w_hdl.close()

The `SeqIO.write()` function takes a **list of sequences to write** (not just a single
one). Be careful with this idiom. 
</br>
If you want to write many sequences (and you
could easily write millions with NGS), do not use a list (as shown in the preceding
code), because **this will allocate massive amounts of memory**.
</br>
 Either use an
iterator, or use the SeqIO.write function several times with a subset of the
sequence on each write.

**How to read FASTA file from local disk?**

In [80]:
recs = SeqIO.parse('example.fasta', 'fasta')
for rec in recs:
    seq = rec.seq
    print(rec.description)
    print(seq[:10])

AACAGTTCCTAGAAAATGGAGCTGTCTTGGCATGTAGTCTTTATTGCCCTGCTAAGTTTTTCATGCTGGGGGTCAGACTGGGAGTCTGATAGAAATTTCATTTCCACCGCTGGTCCTCTAACCAATGACTTGCTGCACAACCTGAGTGGTCTCCTGGGAGACCAGAGTTCTAACTTTGTAGCAGGGGACAAAGACATGTATGTTTGTCACCAGCCACTGCCCACTTTCCTGCCAGAATACTTCAGCAGTCTCCATGCCAGTCAGATCACCCATTATAAGGTATTTCTGTCATGGGCACAGCTCCTCCCAGCAGGAAGCACCCAGAATCCAGACGAGAAAACAGTGCAGTGCTACCGGCGACTCCTCAAGGCCCTCAAGACTGCACGGCTTCAGCCCATGGTCATCCTGCACCACCAGACCCTCCCTGCCAGCACCCTCCGGAGAACCGAAGCCTTTGCTGACCTCTTCGCCGACTATGCCACATTCGCCTTCCACTCCTTCGGGGACCTAGTTGGGATCTGGTTCACCTTCAGTGACTTGGAGGAAGTGATCAAGGAGCTTCCCCACCAGGAATCAAGAGCGTCACAACTCCAGACCCTCAGTGATGCCCACAGAAAAGCCTATGAGATTTACCACGAAAGCTATGCTTTTCAGGGCGGAAAACTCTCTGTTGTCCTGCGAGCTGAAGATATCCCGGAGCTCCTGCTAGAACCACCCATATCTGCGCTTGCCCAGGACACGGTCGATTTCCTCTCTCTTGATTTGTCTTATGAATGCCAAAATGAGGCAAGTCTGCGGCAGAAGCTGAGTAAATTGCAGACCATTGAGCCAAAAGTGAAAGTTTTCATCTTCAACCTAAAACTCCCAGACTGCCCCTCCACCATGAAGAACCCAGCCAGTCTGCTCTTCAGCCTTTTTGAAGCCATAAATAAAGACCAAGTGCTCACCATTGGGTTTGATATTAATGAGTTTCTGAGTTGTTCATCAAGTTCCAAGAAAA

**Now, we can transcribe DNA as follows:**

In [81]:
rna = seq.transcribe()
print(rna)

AACAGUUCCUAGAAAAUGGAGCUGUCUUGGCAUGUAGUCUUUAUUGCCCUGCUAAGUUUUUCAUGCUGGGGGUCAGACUGGGAGUCUGAUAGAAAUUUCAUUUCCACCGCUGGUCCUCUAACCAAUGACUUGCUGCACAACCUGAGUGGUCUCCUGGGAGACCAGAGUUCUAACUUUGUAGCAGGGGACAAAGACAUGUAUGUUUGUCACCAGCCACUGCCCACUUUCCUGCCAGAAUACUUCAGCAGUCUCCAUGCCAGUCAGAUCACCCAUUAUAAGGUAUUUCUGUCAUGGGCACAGCUCCUCCCAGCAGGAAGCACCCAGAAUCCAGACGAGAAAACAGUGCAGUGCUACCGGCGACUCCUCAAGGCCCUCAAGACUGCACGGCUUCAGCCCAUGGUCAUCCUGCACCACCAGACCCUCCCUGCCAGCACCCUCCGGAGAACCGAAGCCUUUGCUGACCUCUUCGCCGACUAUGCCACAUUCGCCUUCCACUCCUUCGGGGACCUAGUUGGGAUCUGGUUCACCUUCAGUGACUUGGAGGAAGUGAUCAAGGAGCUUCCCCACCAGGAAUCAAGAGCGUCACAACUCCAGACCCUCAGUGAUGCCCACAGAAAAGCCUAUGAGAUUUACCACGAAAGCUAUGCUUUUCAGGGCGGAAAACUCUCUGUUGUCCUGCGAGCUGAAGAUAUCCCGGAGCUCCUGCUAGAACCACCCAUAUCUGCGCUUGCCCAGGACACGGUCGAUUUCCUCUCUCUUGAUUUGUCUUAUGAAUGCCAAAAUGAGGCAAGUCUGCGGCAGAAGCUGAGUAAAUUGCAGACCAUUGAGCCAAAAGUGAAAGUUUUCAUCUUCAACCUAAAACUCCCAGACUGCCCCUCCACCAUGAAGAACCCAGCCAGUCUGCUCUUCAGCCUUUUUGAAGCCAUAAAUAAAGACCAAGUGCUCACCAUUGGGUUUGAUAUUAAUGAGUUUCUGAGUUGUUCAUCAAGUUCCAAGAAAA

**Finally, we can translate our gene into a protein:**

In [82]:
prot = seq.translate()
print(prot)

NSS*KMELSWHVVFIALLSFSCWGSDWESDRNFISTAGPLTNDLLHNLSGLLGDQSSNFVAGDKDMYVCHQPLPTFLPEYFSSLHASQITHYKVFLSWAQLLPAGSTQNPDEKTVQCYRRLLKALKTARLQPMVILHHQTLPASTLRRTEAFADLFADYATFAFHSFGDLVGIWFTFSDLEEVIKELPHQESRASQLQTLSDAHRKAYEIYHESYAFQGGKLSVVLRAEDIPELLLEPPISALAQDTVDFLSLDLSYECQNEASLRQKLSKLQTIEPKVKVFIFNLKLPDCPSTMKNPASLLFSLFEAINKDQVLTIGFDINEFLSCSSSSKKSMSCSLTGSLALQPDQQQDHETTDSSPASAYQRIWEAFANQSRAERDAFLQDTFPEGFLWGASTGAFNVEGGWAEGGRGVSIWDPRRPLNTTEGQATLEVASDSYHKVASDVALLCGLRAQVYKFSISWSRIFPMGHGSSPSLPGVAYYNKLIDRLQDAGIEPMATLFHWDLPQALQDHGGWQNESVVDAFLDYAAFCFSTFGDRVKLWVTFHEPWVMSYAGYGTGQHPPGISDPGVASFKVAHLVLKAHARTWHHYNSHHRPQQQGHVGIVLNSDWAEPLSPERPEDLRASERFLHFMLGWFAHPVFVDGDYPATLRTQIQQMNRQCSHPVAQLPEFTEAEKQLLKGSADFLGLSHYTSRLISNAPQNTCIPSYDTIGGFSQHVNHVWPQTSSSWIRVVPWGIRRLLQFVSLEYTRGKVPIYLAGNGMPIGESENLFDDSLRVDYFNQYINEVLKAIKEDSVDVRSYIARSLIDGFEGPSGYSQRFGLHHVNFSDSSKSRTPRKSAYFFTSIIEKNGFLTKGAKRLLPPNTVNLPSKVRAFTFPSEVPSKAKVVWEKFSSQPKFERDLFYHGTFRDDFLWGVSSSAYQIEGAWDADGKGPSIWDNFTHTPGSNVKDNATGDIACDSYHQLDADLNMLRALKVKAYRFSISWSRIFPTGRNSSINSH